In [1]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Tue_Aug_15_22:02:13_PDT_2023
Cuda compilation tools, release 12.2, V12.2.140
Build cuda_12.2.r12.2/compiler.33191640_0


In [2]:
!pip install git+https://github.com/andreinechaev/nvcc4jupyter.git


  Cloning https://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-1ghsxmev
  Running command git clone --filter=blob:none --quiet https://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-1ghsxmev
  Resolved https://github.com/andreinechaev/nvcc4jupyter.git to commit 0d2ab99cccbbc682722e708515fe9c4cfc50185a
  Preparing metadata (setup.py) ... done
  Created wheel for NVCCPlugin: filename=NVCCPlugin-0.0.2-py3-none-any.whl size=4716 sha256=61e146f6e92ff2f6f0cdb3d44566210c3f7c7b9fe6a231f2926c4c783057537a
  Stored in directory: /tmp/pip-ephem-wheel-cache-oe46da31/wheels/a8/b9/18/23f8ef71ceb0f63297dd1903aedd067e6243a68ea756d6feea
Successfully built NVCCPlugin


In [3]:
	%load_ext nvcc_plugin


created output directory at /content/src
Out bin /content/result.out


In [4]:
%%cu
#include <iostream>

__global__ void add(int *x, int *y, int *z){
    int id = threadIdx.x;
    z[id] = x[id] + y[id];
}

int main(){

    // host variables
    int a[5] = {1, 2, 3, 4, 5};
    int b[5] = {2, 3, 4, 5, 6};
    int c[5];

    // device variables
    int *d, *e, *f;

    // memory allocation on the device
    cudaMalloc((void **) &d, 5 * sizeof(int));
    cudaMalloc((void **) &e, 5 * sizeof(int));
    cudaMalloc((void **) &f, 5 * sizeof(int));

    // copy input data to the device
    cudaMemcpy(d, &a, 5 * sizeof(int), cudaMemcpyHostToDevice);
    cudaMemcpy(e, &b, 5 * sizeof(int), cudaMemcpyHostToDevice);

    // launch kernel
    int block_num = 1;
    int threads_per_block = 6;
    add<<<block_num, threads_per_block>>>(d, e, f);
    cudaDeviceSynchronize();

    // copy data from device to host
    cudaMemcpy(&c, f, 5*sizeof(int), cudaMemcpyDeviceToHost);

    // print result
    printf("Result: ");
    for(int i = 0; i < 5; i++){
        printf("%d ", c[i]);
    }

    // free memmory allocated to pointers
    cudaFree(d);
    cudaFree(e);
    cudaFree(f);

    return 0;
}

Result: 3 5 7 9 11 
